# Pyomo

[![Index](https://img.shields.io/badge/Index-blue)](../index.ipynb)
[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/digillia/Digillia-Colab/blob/main/tools/pyomo.ipynb)

Comme [OR Tools](https://github.com/google/or-tools), [Pyomo](https://github.com/Pyomo/pyomo) permet de formuler facilement des problèmes d'optimisation mathématique en Python et de transmettre ces problèmes à un solveur. Pyomo peut utiliser de nombreux solveurs : [glpk](https://www.gnu.org/software/glpk/) est gratuit et pratique pour les problèmes d'optimisation de petite et moyenne taille.

Docs:
- https://github.com/Pyomo/pyomo
- https://www.pyomo.org/
- https://www.gnu.org/software/glpk/

Voir aussi:
- https://jckantor.github.io/ND-Pyomo-Cookbook/README.html

In [17]:
import sys

# Supprimer les commentaires pour installer
# !pip3 install -q -U numpy
# !pip3 install -q -U pandas

# À installer dans tous les cas pour Google Colab
if 'google.colab' in sys.modules:
    !pip3 install -q -U pyomo

In [18]:
if sys.platform == 'linux' or sys.platform == 'linux2':
    !apt-get install -y -qq glpk-utils
elif sys.platform == 'darwin':
    # Assurez-vous d'avoir brew installé: https://brew.sh/
    !brew install glpk
elif sys.platform == 'win32':
    print('Download and run the installer from https://winglpk.sourceforge.net/')

# Vérifier l'installation du solveur glpk
# !glpsol --help

executable = 'glpsol' #'/usr/bin/glpsol'

==> Downloading https://formulae.brew.sh/api/formula.jws.json

==> Downloading https://formulae.brew.sh/api/cask.jws.json

To reinstall 5.0, run:
  brew reinstall glpk


In [19]:
import numpy as np
import pandas as pd
from pyomo.environ import *

## Premier problème d'optimisation linéaire

Un fabricant d'électroménager tente de maximiser ses profits en décidant du nombre de lave-linges et de sèche-linge à produire, sous réserve de contraintes sur les ressources de fabrication, de test et d'assemblage:
- Les ressources de l’entreprise sont telles que seulement 20 heures de fabrication, 20 heures d’assemblage et 25 heures de tests sont disponibles par jour.
- La production d'un lave-linge nécessite 1 heure de fabrication, 2 heures d'assemblage et 2 heures de tests pour générer un bénéfice de €100,00.
- La production d'un sèche-linge nécessite 2 heures de fabrication, 1 heure d'assemblage et 2 heures de tests pour générer un bénéfice de €120,00.

Nous avons formulé mathématiquement ce problème comme suit, en définissant
- $x_1$ comme le nombre de lave-linges produits par jour, et
- $x_2$ comme le nombre de sèche-linges produits par jour.

$$
\begin{aligned}
& \underset{x}{\text{maximiser}}
& & 100 x_1 + 120 x_2\\
& \text{sachant que}
& & x_1 + 2 x_2 \leq 20 \\
& & & 2x_1 +  x_2 \leq 20 \\
& & & 2x_1 + 2 x_2 \leq 25 \\
&&& x_1, x_2 \geq 0
\end{aligned}
$$

In [20]:
# Création du modèle
model = ConcreteModel()

# Déclaration des 2 variables x1 et x2
model.x = Var([1,2], domain=NonNegativeReals)

# Déclaration de l'objectif
model.profit = Objective(expr = 100*model.x[1] + 120*model.x[2], sense=maximize)

# Déclaration des contraintes
model.manufacturing = Constraint(expr = model.x[1] +2*model.x[2] <= 20)
model.assembly = Constraint(expr = 2*model.x[1] + model.x[2] <= 20)
model.testing = Constraint(expr = 2*model.x[1] + 2*model.x[2] <= 25)

# Affichage du modèle
model.pprint()

1 Var Declarations
    x : Size=2, Index={1, 2}
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :  None :  None : False :  True : NonNegativeReals
          2 :     0 :  None :  None : False :  True : NonNegativeReals

1 Objective Declarations
    profit : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 100*x[1] + 120*x[2]

3 Constraint Declarations
    assembly : Size=1, Index=None, Active=True
        Key  : Lower : Body          : Upper : Active
        None :  -Inf : 2*x[1] + x[2] :  20.0 :   True
    manufacturing : Size=1, Index=None, Active=True
        Key  : Lower : Body          : Upper : Active
        None :  -Inf : x[1] + 2*x[2] :  20.0 :   True
    testing : Size=1, Index=None, Active=True
        Key  : Lower : Body            : Upper : Active
        None :  -Inf : 2*x[1] + 2*x[2] :  25.0 :   True

5 Declarations: x profit manufacturing assembly testing


In [21]:
# Résolution du modèle avec le solveur glpk
SolverFactory('glpk', executable=executable).solve(model).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 1400.0
  Upper bound: 1400.0
  Number of objectives: 1
  Number of constraints: 3
  Number of variables: 2
  Number of nonzeros: 6
  Sense: maximize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.05493307113647461
# ----------------------------------------------------------
#   Solution Information
# -----------------------------------

In [22]:
# Affichage de la solution
def print_solution(model):
    print('Profit quotidien = ', model.profit())
    print('Nombre de lave-linges  x1 = ', model.x[1].value)
    print('Nombre de sèche-linges x2 = ', model.x[2].value)
print_solution(model)

Profit quotidien =  1400.0
Nombre de lave-linges  x1 =  5.0
Nombre de sèche-linges x2 =  7.5


In [23]:
# Affichage des ressources utilisées
def print_resources(model):
    print('Ressources utilisées')
    print('Fabrication  = ', model.manufacturing())
    print('Assemblage = ', model.assembly())
    print('Test = ', model.testing())
print_resources(model)

Ressources utilisées
Fabrication  =  20.0
Assemblage =  17.5
Test =  25.0


On note que la production est limitée par la fabrication et le test, puisque les 20 heures d'assemblage disponibles quotidiennement ne sont pas utilisées.

In [24]:
# Désactivation des contraintes limitatives
model.manufacturing.deactivate()
model.testing.deactivate()

# Augmentation des ressources disponibles
model.manufacturing_new = Constraint(expr = model.x[1] +2*model.x[2] <= 21 )
model.testing_new = Constraint(expr = 2*model.x[1] +2*model.x[2] <= 26 )

# Résolution du modèle avec le solveur glpk
SolverFactory('glpk', executable=executable).solve(model)

# Affichage la solution
print_solution(model)
print('\n')
# Affichage des ressources utilisées
print_resources(model)

Profit quotidien =  1460.0
Nombre de lave-linges  x1 =  5.0
Nombre de sèche-linges x2 =  8.0


Ressources utilisées
Fabrication  =  21.0
Assemblage =  18.0
Test =  26.0


## Deuxième problème d'optimisation linéaire

Un distributeur cherche à optimiser la distribution:
- dans un ensemble de villes $C$ où des entrepôts peuvent être établis,
- dans un ensemble de régions $R$ à approvisionner,
- avec des coûts fixes $c_i$ pour l'ouverture d'un entrepôt dans la ville $i$,
- avec une demande $b_j$ de chaque région $j$, et
- avec des frais d'expédition $t_{ij}$ de la ville $i$ à la région $j$.

L'optimisation consiste à décider des variables
- $y_i$ (1 ou 0 - pour ouvrir ou pas un entrepôt dans la ville $i$), et
- $x_{ij}$ (quelle quantité de fournitures envoyer de la ville $i$ à la région $j$ sachant que chaque entrepôt a une capacité limitée à 100).

Nous avons formulé mathématiquement ce problème d'optimisation comme suit (toutes les données ont une périodicité hebdomadaire):

$$
\begin{aligned}
& \underset{x,y}{\text{minmiser}}
& & \sum_{i \in C}c_i y_i + \sum_{i \in C}\sum_{j \in R} t_{ij} x_{ij}\\
& \text{sachant que}
& & \sum_{j \in R} x_{ij} \leq 100 y_i, \forall \:  i \in C \\
& & & \sum_{i \in C} x_{ij} = b_j, \forall \:  j \in R \\
& & & x_{ij} \in \mathbb{Z}^+  , \forall \:  i \in C, j \in R  \\
&&& y_i \in \{0,1\} , \forall \:  i \in C
\end{aligned}
$$

In [25]:
# Les villes des entrepôts
C = range(4)

# Les régions désservies
R = range(3)

# Les coûts fixes correspondants
c = [400, 500, 300, 150]

# La demande dans chaque région
b = [80, 70, 40]

# Les frais d'expédition de ville (entrepôt) à région
df = pd.DataFrame.from_dict({
    'City': C,
    'Region 1': [20, 48, 26, 24],
    'Region 2': [40, 15, 35, 50],
    'Region 3': [50, 26, 18, 35]
})
t = np.array(df.iloc[0:4,1:4])
df

,City,Region 1,Region 2,Region 3
0,0,20,40,50
1,1,48,15,26
2,2,26,35,18
3,3,24,50,35


In [26]:
# Création du model
model = ConcreteModel()

# Déclaration des variables y_i et x_ij
model.y = Var(C, domain=Binary)
model.x = Var(C, R, domain=NonNegativeIntegers)

# Déclaratin de l'objectif
model.profit = Objective(expr =  sum(c[i]*model.y[i] for i in C) + 
                         sum(sum(t[i,j]*model.x[i,j] for i in C) for j in R ), sense=minimize)

# Déclaration des contraintes
model.constraints = ConstraintList()
#forcing constraints
for i in C:
    model.constraints.add(sum(model.x[i,j] for j in R) <= 100*model.y[i])
# Demande des régions
for j in R:
    model.constraints.add(sum(model.x[i,j] for i in C) == b[j])

In [27]:
# Résolution du modèle avec le solveur glpk
SolverFactory('glpk', executable=executable).solve(model).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 4570.0
  Upper bound: 4570.0
  Number of objectives: 1
  Number of constraints: 7
  Number of variables: 16
  Number of nonzeros: 28
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 5
      Number of created subproblems: 5
  Error rc: 0
  Time: 0.006451129913330078
# ----------------------------------------------------------
#   Solution Information
# --------------------------------

In [28]:
# Affichage de la solution 
print('Profit = ', model.profit())
print('\n')
print('y: entrepôts ouverts')
print(pd.DataFrame({'Ville':df.City,'Ouvert?':[('oui' if model.y[i].value == 1 else 'non') for i in C]}))
print('\n')
print('x: flux de la ville i vers la région j')
x = df.copy()
for i in C:
  for j in R:
    x.iloc[i,j+1] = model.x[i,j].value
print(x)

Profit =  4570.0


y: entrepôts ouverts
   Ville Ouvert?
0      0     oui
1      1     oui
2      2     oui
3      3     non


x: flux de la ville i vers la région j
   City  Region 1  Region 2  Region 3
0     0        80         0         0
1     1         0        70         0
2     2         0         0        40
3     3         0         0         0
